In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/industrial-residential-air-quality-classification/City_Types.csv


In [2]:
df=pd.read_csv('/kaggle/input/industrial-residential-air-quality-classification/City_Types.csv')

In [3]:
df.head(5)

,Date,City,CO,NO2,SO2,O3,PM2.5,PM10,Type
0,2024-01-01 00:00:00+00:00,Moscow,208.0,15.9,13.2,44.0,8.6,9.4,Industrial
1,2024-01-01 01:00:00+00:00,Moscow,207.0,17.4,13.7,44.0,8.6,10.5,Industrial
2,2024-01-01 02:00:00+00:00,Moscow,217.0,19.0,15.5,43.0,10.4,12.9,Industrial
3,2024-01-01 03:00:00+00:00,Moscow,231.0,21.0,20.7,36.0,12.3,15.3,Industrial
4,2024-01-01 04:00:00+00:00,Moscow,263.0,34.5,27.2,27.0,13.6,20.0,Industrial


In [4]:
df['Date']=pd.to_datetime(df['Date'])

In [5]:
df['Month']=df['Date'].dt.month
df['Day']=df['Date'].dt.dayofweek

In [6]:
df.head(5)

,Date,City,CO,NO2,SO2,O3,PM2.5,PM10,Type,Month,Day
0,2024-01-01 00:00:00+00:00,Moscow,208.0,15.9,13.2,44.0,8.6,9.4,Industrial,1,0
1,2024-01-01 01:00:00+00:00,Moscow,207.0,17.4,13.7,44.0,8.6,10.5,Industrial,1,0
2,2024-01-01 02:00:00+00:00,Moscow,217.0,19.0,15.5,43.0,10.4,12.9,Industrial,1,0
3,2024-01-01 03:00:00+00:00,Moscow,231.0,21.0,20.7,36.0,12.3,15.3,Industrial,1,0
4,2024-01-01 04:00:00+00:00,Moscow,263.0,34.5,27.2,27.0,13.6,20.0,Industrial,1,0


In [7]:
df.drop(columns=['Date'],inplace=True)

In [8]:
df.head(5)

,City,CO,NO2,SO2,O3,PM2.5,PM10,Type,Month,Day
0,Moscow,208.0,15.9,13.2,44.0,8.6,9.4,Industrial,1,0
1,Moscow,207.0,17.4,13.7,44.0,8.6,10.5,Industrial,1,0
2,Moscow,217.0,19.0,15.5,43.0,10.4,12.9,Industrial,1,0
3,Moscow,231.0,21.0,20.7,36.0,12.3,15.3,Industrial,1,0
4,Moscow,263.0,34.5,27.2,27.0,13.6,20.0,Industrial,1,0


In [9]:
df['City'].unique()

array(['Moscow', 'Delhi', 'Beijing', 'Zurich', 'Vancouver', 'Stockholm'],
      dtype=object)

In [10]:
df=pd.get_dummies(df,columns=['City'])

In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Type']=le.fit_transform(df['Type'])

In [12]:
df.head(3)

,CO,NO2,SO2,O3,PM2.5,PM10,Type,Month,Day,City_Beijing,City_Delhi,City_Moscow,City_Stockholm,City_Vancouver,City_Zurich
0,208.0,15.9,13.2,44.0,8.6,9.4,0,1,0,False,False,True,False,False,False
1,207.0,17.4,13.7,44.0,8.6,10.5,0,1,0,False,False,True,False,False,False
2,217.0,19.0,15.5,43.0,10.4,12.9,0,1,0,False,False,True,False,False,False


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52704 entries, 0 to 52703
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CO              52704 non-null  float64
 1   NO2             52704 non-null  float64
 2   SO2             52704 non-null  float64
 3   O3              52704 non-null  float64
 4   PM2.5           52704 non-null  float64
 5   PM10            52704 non-null  float64
 6   Type            52704 non-null  int64  
 7   Month           52704 non-null  int32  
 8   Day             52704 non-null  int32  
 9   City_Beijing    52704 non-null  bool   
 10  City_Delhi      52704 non-null  bool   
 11  City_Moscow     52704 non-null  bool   
 12  City_Stockholm  52704 non-null  bool   
 13  City_Vancouver  52704 non-null  bool   
 14  City_Zurich     52704 non-null  bool   
dtypes: bool(6), float64(6), int32(2), int64(1)
memory usage: 3.5 MB


In [14]:
df.duplicated().sum()

1

In [15]:
df=df.drop_duplicates()

In [16]:
X=df.drop(columns=['Type'])
y=df['Type']

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [18]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train_scaled=sc.fit_transform(X_train)
X_test_scaled=sc.transform(X_test)

In [19]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

2025-07-08 17:29:54.404027: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751995794.655343      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751995794.727550      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [20]:
model=Sequential()
model.add(Dense(10,activation='relu',input_dim=14))
model.add(Dense(10,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-08 17:30:09.463730: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 10)                  │             150 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             110 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 271 (1.06 KB)

 Trainable params: 271 (1.06 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [23]:
from keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True,verbose=1)

In [24]:
history=model.fit(X_train_scaled,y_train,epochs=50,validation_split=0.2,callbacks=[early_stop])

Epoch 1/50
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8583 - loss: 0.2672 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 2/50
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 7.1525e-04 - val_accuracy: 1.0000 - val_loss: 2.1893e-04
Epoch 3/50
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 1.6734e-04 - val_accuracy: 1.0000 - val_loss: 7.9453e-05
Epoch 4/50
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 6.4803e-05 - val_accuracy: 1.0000 - val_loss: 3.5279e-05
Epoch 5/50
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 2.9815e-05 - val_accuracy: 1.0000 - val_loss: 1.7304e-05
Epoch 6/50
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 1.4871e-05 - val_accuracy: 1.0000 - val_loss: 8.9263e-06
Epoch 7/50
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 7.9530e-06 - val_accuracy: 1.0000 - val_loss: 4.7407e-06
Epoch 8/50
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 

In [25]:
y_log=model.predict(X_test_scaled)
y_pred=np.where(y_log>0.5,1,0)

330/330 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [26]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

1.0

In [27]:
y_test.value_counts()

Type
0    5293
1    5248
Name: count, dtype: int64

In [28]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))


[[5293    0]
 [   0 5248]]


In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5293
           1       1.00      1.00      1.00      5248

    accuracy                           1.00     10541
   macro avg       1.00      1.00      1.00     10541
weighted avg       1.00      1.00      1.00     10541

